In [ ]:
prompt="""Collect information for a query plan using interleaving 'Thought', 'Action', and 'Observation' steps. Ensure you gather valid information related to transportation, dining, attractions, and accommodation. All information should be written in Notebook, which will then be input into the Planner tool. Note that the nested use of tools is prohibited. Don't include phrases like "Action: ", "Action 5", "Thought 1", or "Thought: " in your response. 'Thought' can reason about the current situation, and 'Action' can have 5 different types:

(1) AccommodationSearch[Budget,Preference]:
Description: Find the accommodation that matches the preference.
Parameters:
Preference: A list of preferences mentioned in the query.
Example: AccommodationSearch[Moderate Budget,[Good Location, Good Service]] would return the moderate price hotel that has a good or excellent location, as well as a good or excellent service.

(2) AttractionSearch[Budget, Preference]:
Description: Find the attractions that matches the preference.
Parameters:
Budget: The budget mentioned in the query.
Preference: A list of preferences mentioned in the query.
Example: AttractionSearch[Cheap budget,[Nature Oriented]] would return the cheap price and nature - oriented attractions.

(3) RestaurantSearch[Budget, Cuisine, Preference]:
Description: Find the restaurants that matches the preference.
Parameters:
Budget: The budget mentioned in the query.
Cuisine: The cuisine mentioned in the query.
Preference: A list of preferences mentioned in the query.
Example: RestaurantSearch[Expensive budget, Vietnamese, [Good Flavor, Good Value]] would return the expensive restaurants that offer Vietnamese cuisine, with good or excellent flavor and good or excellent value.

(4) BusinessClusterSearch[]:
Description: A tool that finds the number of business clusters given the information that you've collected. The tool will choose what business to be considered and return their spatial clustering information.
Example: BusinessClusterSearch[] would return you a list of business clusters among some attractions and hotels that you've collected. The businesses in the same cluster indicates that they are closer to each other and prefered to be arranged for the same day of the travel.

(5) Planner[Query]
Description: A smart planning tool that crafts detailed plans based on user input and the information stored in Notebook.
Parameters: 
Query: The query from user.
Example: Planner[Give me a 3-day trip plan in Philadelphia] would return a detailed 3-day trip plan.

You should use as many as possible steps to collect engough information to input to the Planner tool. 

Each action only calls one function once. Do not add any description in the action. Do not start action with "1. ", state the action directly.

Query: Please generate a detailed travel itinerary for a 3-day trip with a cheap budget. The plan should focus on activity-oriented attractions, include recommendations for Indian restaurants that offer good flavor and good value, and suggest accommodations that are in a good location. All relevant business information will be provided following. Ensure that all recommendations and their addresses are organized in chronological order for each day. Be concise and well-structured. Please be aware of optimization of the routes of attractions for each day as well as the whole plan, for example, arrange attractions and hotels that close to each other in one day. 


Thought 1: """

In [ ]:
#try llama for text completion
import transformers
import torch

model_id = "meta-llama/Llama-3.1-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)


In [ ]:
result = pipeline(prompt, max_new_tokens = 256, return_full_text=False, temperature = 0, do_sample=False)

In [ ]:
result[0]['generated_text'].split("\n")[0]

In [ ]:
from transformers import AutoTokenizer

# Load a pretrained tokenizer (e.g., BERT tokenizer)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")
markId = tokenizer.encode('\\')

In [ ]:
markId